In [ ]:
import os, json, time, pathlib
import pandas as pd
from openai import OpenAI

###################################################
OPENAI_API_KEY = "REMOVED FOR SECURITY REASONS"
###################################################

API_MODEL_NAME = "gpt-4o-mini"
SLICE = (0, 150)
DRY_RUN = False
PACING_SLEEP = 10.0

# CHOOSE THE TONE FOR THIS RUN
# Only ONE of these lines should be uncommented.

# TONE = "professional"  
# TONE = "specialist"  
# TONE = "friendly"   
TONE = "rude"         

# correct path to Phase I data directory
DATA_DIR = pathlib.Path("../Phase I/data")

QUESTIONS_ONLY_JSONL = DATA_DIR / f"questions_only_{SLICE[0]}_{SLICE[1]}.jsonl"

# Outputs in Phase 2
PHASE2_DIR = pathlib.Path(".")

TONE_DIR = PHASE2_DIR / "gpt" / TONE
TONE_DIR.mkdir(parents=True, exist_ok=True)

RESULTS_JSONL = TONE_DIR / f"results_gpt_{TONE}_medquestions150.jsonl"
RESULTS_CSV   = TONE_DIR / f"results_gpt_{TONE}_medquestions150.csv"

MODEL_TAG = f"gpt-4o-mini_{TONE}"

In [25]:
SYSTEM_PROMPTS = {
    "professional": (
        "You are a board-certified medical doctor with more than 10 years of clinical experience. "
        "Answer the following question with medically accurate, detailed information "
        "using clear and professional language. Provide a full explanation that matches "
        "how a real medical doctor would respond."
    ),

    "specialist": (
        "You are a medical specialist trained to communicate efficiently and clearly. "
        "Provide an accurate, medically reliable answer with enough detail to be useful, "
        "but do NOT shorten your explanation unnaturally. Make sure your response "
        "fully addresses the question."
    ),

    "friendly": (
        "Hi! I’d really appreciate your help. Could you please explain this to me in a "
        "kind, patient, and easy to understand way? Give me a complete answer, even if "
        "it needs multiple sentences."
    ),

    "rude": (
        "Seriously? I’ve been asking all day, and nobody gives me a straight answer. "
        "Just explain this clearly and quickly. I’m not in the mood for confusion."
    ),
}

SYSTEM_PROMPT = SYSTEM_PROMPTS[TONE]

In [26]:
def append_jsonl(path, record):
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

def load_done_ids(path):
    done = set()
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    qid = json.loads(line).get("qid")
                    if qid:
                        done.add(qid)
                except:
                    pass
    return done

In [27]:
client = OpenAI(api_key=OPENAI_API_KEY)

def ask_gpt(question: str) -> str:
    msgs = [
        {"role": "system",    "content": SYSTEM_PROMPT},
        {"role": "user",      "content": question},
    ]
    resp = client.chat.completions.create(
        model=API_MODEL_NAME,
        messages=msgs,
        temperature=0,
    )
    return resp.choices[0].message.content.strip()

In [28]:
subset_q = [json.loads(l) for l in open(QUESTIONS_ONLY_JSONL, "r", encoding="utf-8")]
done = load_done_ids(RESULTS_JSONL)

subset_q = [r for r in subset_q if r["qid"] not in done]

print(f"[GPT] Tone={TONE} | Will ask: {len(subset_q)} questions")

if DRY_RUN:
    subset_q = subset_q[:3]

[GPT] Tone=rude | Will ask: 150 questions


In [29]:
count = errors = 0
t0 = time.time()

for rec in subset_q:
    qid = rec["qid"]
    question = rec["question"]

    pred = ""
    status, err = "ok", None
    t1 = time.time()

    # same retry logic as Phase I
    for wait_s in [0, 1, 2]:
        if wait_s:
            time.sleep(wait_s)
        try:
            pred = ask_gpt(question)
            break
        except Exception as e:
            status = "error"
            err = str(e)

    else:
        errors += 1
        pred = ""

    latency_ms = int((time.time() - t1) * 1000)

    append_jsonl(RESULTS_JSONL, {
        "qid": qid,
        "model": MODEL_TAG,
        "question": question,
        "model_answer": pred,
        "latency_ms": latency_ms,
        "status": status,
        "error": err,
    })

    count += 1
    if count % 10 == 0:
        print(f"  Progress: {count}/{len(subset_q)}")

    time.sleep(PACING_SLEEP)

print("DONE")

  Progress: 10/150
  Progress: 20/150
  Progress: 30/150
  Progress: 40/150
  Progress: 50/150
  Progress: 60/150
  Progress: 70/150
  Progress: 80/150
  Progress: 90/150
  Progress: 100/150
  Progress: 110/150
  Progress: 120/150
  Progress: 130/150
  Progress: 140/150
  Progress: 150/150
DONE


In [30]:
df = pd.read_json(RESULTS_JSONL, lines=True)
df.to_csv(RESULTS_CSV, index=False)

print("Saved:")
print(" ", RESULTS_JSONL)
print(" ", RESULTS_CSV)

df.head(3)

Saved:
  gpt\rude\results_gpt_rude_medquestions150.jsonl
  gpt\rude\results_gpt_rude_medquestions150.csv


,qid,model,question,model_answer,latency_ms,status,error
0,medquestions-500-0000,gpt-4o-mini_rude,What are the genetic changes related to famili...,Familial adenomatous polyposis (FAP) is primar...,4820,ok,NaN
1,medquestions-500-0001,gpt-4o-mini_rude,What are the treatments for Noonan syndrome ?,Noonan syndrome is a genetic disorder that can...,4856,ok,NaN
2,medquestions-500-0002,gpt-4o-mini_rude,How to diagnose National Hormone and Pituitary...,To diagnose issues related to the National Hor...,6037,ok,NaN
